In [1]:
import csv
import math

#read optimal cirle packing arrangements and used battery types
N = list()
ratio = list()
with open('arrangements.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';', quotechar='|')
    for row in reader:
        N.append(int(row['N']))
        ratio.append(float(row['ratio']))

batteries = list()
with open('batteries.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';', quotechar='|')
    for row in reader:
        batteries.append(row)

In [6]:
#hull sizes [mm]: d - diameter; l - length
hull_ds = list(range(10, 1010, 10))
hull_ls = list(range(10, 1010, 10))

with open('output.csv', 'w', newline='') as csvfile:
    fieldnames = ['hull_D', 'hull_L', 'batt_MODEL', 'batt_D', 'batt_L', 'total_N', 'total_CAP', 'total_MASS']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', quotechar='|')
    writer.writeheader()
    # sweep thru params
    for hull_d in hull_ds:
        for hull_l in hull_ls:
            for model, b in enumerate(batteries):
                batt_d = float(b['D'])
                batt_l = float(b['L'])
                batt_m = float(b['M'])
                batt_c = float(b['C'])
                # hull size to battery size ratio - it gives the optimal packing parameter
                curr_ratio = hull_d / batt_d
                # find closest ratio that is still feasible
                curr_N = 0
                if hull_d >= batt_d:
                    for i in range(len(N)):
                        if ratio[i] > curr_ratio:
                            curr_N = i
                            break
                # calculate the total number of battery cells based on hull length
                N_total = curr_N * math.floor(hull_l / batt_l)
                if N_total == 0:
                    continue
                # if we have >0 batteries, calculate total capacity and mass
                MASS_total = N_total * batt_m
                CAP_total = N_total * batt_c
                # append line to the results
                writer.writerow({'hull_D': hull_d / 1000.0,
                                 'hull_L': hull_l / 1000.0,
                                 'batt_MODEL': model,
                                 'batt_D': batt_d / 1000.0,
                                 'batt_L': batt_l / 1000.0,
                                 'total_N': N_total,
                                 'total_CAP': CAP_total / 1000.0,
                                 'total_MASS': MASS_total})
                # debug output here
                #print(hull_d, hull_l, model, batt_d, batt_l, N_total, MASS_total, CAP_total)